# Churn Predition - Sparkify Project
This project build a classification model to predict user churn analysing user activities on a music streaming app called Sparkify.

The project is build using Pyspark and pass through the steps below:

- Load data to SparkSession
- Clean and Preprocess Data
- Exploratory Data Analysis - EDA
- Feature Selection
- Model building and optimization
- Model evalution

The analysis is initialy done on a tiny subset (128MB) of the full dataset available (12GB). 

Once the initial analysis is done, the full pipeline with all the dataset (12GB) is processed on AWS EMR Spark Cluster. 

In [1]:
import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf, lag
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import countDistinct
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.window import Window

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

import re


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1620854706714_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'pandas'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'pandas'



In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify-flavio4") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark.version

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2.4.3'

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. 

In [4]:
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
#event_data = "s3n://udacity-dsnd/sparkify/mini_sparkify_event_data.json"
df = spark.read.json(event_data)
df.persist()
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Popol Vuh', auth='Logged In', firstName='Shlok', gender='M', itemInSession=278, lastName='Johnson', length=524.32934, level='paid', location='Dallas-Fort Worth-Arlington, TX', method='PUT', page='NextSong', registration=1533734541000, sessionId=22683, song='Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1749042')

In [5]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [6]:
print('total number of rows:' , df.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

total number of rows: 26259199

In [7]:
# clean rows with empty userId
df = df.filter("userId <> ''")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# number of distinct users
n_users = df.select("userId").distinct().count()

print("number of distinct users: ", n_users)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

number of distinct users:  22278

# Exploratory Data Analysis

In [9]:
# type of subscription
df.select("level").distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+
|level|
+-----+
| free|
| paid|
+-----+

In [10]:
# type of page that user visit
df.select("page").distinct().show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+
|page                     |
+-------------------------+
|Cancel                   |
|Submit Downgrade         |
|Thumbs Down              |
|Home                     |
|Downgrade                |
|Roll Advert              |
|Logout                   |
|Save Settings            |
|Cancellation Confirmation|
|About                    |
|Submit Registration      |
|Settings                 |
|Login                    |
|Register                 |
|Add to Playlist          |
|Add Friend               |
|NextSong                 |
|Thumbs Up                |
|Help                     |
|Upgrade                  |
+-------------------------+
only showing top 20 rows

In [11]:
df.filter("userId = '100010'").select(["userId", "page", "artist"]).sort(["ts"]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----+------+
|userId|page|artist|
+------+----+------+
+------+----+------+

In [12]:
# user location
print('number of disctinct locations: ',
      df.select('location').distinct().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

number of disctinct locations:  887

### Define Churn

A column `Churned` is created to identify users that has churned. 
`Cancellation Confirmation` events is used to define churn, which happen for both paid and free users.

In [13]:
# define user churn
hasCancelled = udf(lambda x: 1 if x=='Cancellation Confirmation' else 0, IntegerType())
df = df.withColumn('Churn', hasCancelled(df['page']))

user_window = Window.partitionBy('userId')
df = df.withColumn('Churned', F.max('Churn').over(user_window))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### calculate churn rate

In [14]:
n_user_churn = df.select(['userId']).filter("Churn=1").distinct().count()

print("total users: ", n_users)
print("users churned: ",n_user_churn)
print("users not churned: ", (n_users - n_user_churn) )
print("churn rate: ", n_user_churn/n_users )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

total users:  22278
users churned:  5003
users not churned:  17275
churn rate:  0.22457132597181076

### Explore Data
Once defined churn, we perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. 

## Feature Engineering

New features are created based on the original dataset 

In [15]:
# create new column to count the number of songs played by the user
is_song = udf(lambda x: 1 if x=='NextSong' else 0, IntegerType())
df = df.withColumn('NextSong', is_song(df['page']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# create new column to check if user has been in paid subscription
has_paid= udf(lambda x: 1 if x=='paid' else 0, IntegerType()) 
df = df.withColumn('hasPaid', has_paid('level'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# create new column to check if user has downgrade
has_downgrade =  udf(lambda x: 1 if x=='Downgrade' else 0, IntegerType()) 
df = df.withColumn('hasDowngrade', has_downgrade('page'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### general info aggregated by userId

In [18]:
# general info aggregated by userId
user = df.groupBy("userId").agg(
    F.max('location').alias('max_location'),
    F.max('gender').alias('max_gender'),
    F.max('hasPaid').alias('max_hasPaid'),
    F.max('hasDowngrade').alias('max_hasDowngrade'),
    F.max('Churn').alias('max_churn'),
    F.countDistinct('artist').alias('dist_artist')
#    F.collect_list("artist").alias("artist_list")
)

#user.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### time between user sessions

In [19]:
# time between sessions

# rank in session
session_by_ts_win = Window.partitionBy(['userId','sessionId']).orderBy('ts')
df = df.withColumn('rank', F.rank().over(session_by_ts_win))

# time_diff
user_by_ts = Window.partitionBy(['userId']).orderBy('ts')
df = df.withColumn('ts_diff', col('ts') - F.lag('ts',1).over(user_by_ts))

# time diff between first page in current session - last page in previous session
time_btw_session = df.select(['userId','ts_diff'])\
                         .filter("rank=1 and ts_diff is not null")\
                         .groupBy('userId').agg(avg('ts_diff').alias('time_btw_sessions'))
#time_btw_session.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### number of songs played, and number os sessions aggregated BY user, session


In [20]:
# number of songs played, and number os sessions
song_session = df.groupBy(['userId','sessionId'])\
                .agg(
                    F.sum('length'), 
                    F.sum('NextSong')\
                )\
                .groupBy('userId')\
                .agg(
                    F.avg('sum(length)').alias("avg_session_length"),
                    F.avg('sum(NextSong)').alias("avg_session_songs"),
                    F.sum('sum(length)').alias("total_session_length"),   
                    F.sum('sum(NextSong)').alias("total_songs_played"),
                    F.count('sum(length)').alias("total_sessions")
                )

#song_session.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### join dataframes with aggregated columns

This step joins 3 different dataframes that needed to be computed in separated steps:
- data aggregated at the user level
- data aggreated at the user and session level
- time between session

In [21]:
joined_df = None
joined_df = user.join(song_session, ["userId"], "inner")\
                .join(time_btw_session, ["userId"], "inner")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Persist processed dataframe and save to local file or Amazon S3
To this point, all the preprecessing steps has been concluded. (cleaning, aggregation)

Aggregated data will be stored to the local file system, or remote stored like Amazon S3.

Thosee steps save computation time and resources, as all the followings steps (analysis, modelling) is done on aggregated info.

In [22]:
user2 = joined_df.persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
user2.write.mode("overwrite").parquet("joined_df2.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Modeling

- Split the full dataset into train and test
- Test out several of the machine learning (LogisticRegression, RandomForest, LinearSVC)
- Evaluate the performance of various models, tuning parameters as necessary. 
- Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.
- Determine your winning model based on testset

In [25]:
# loading cleaned and aggregated data
df2 = spark.read.parquet("joined_df2.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# define label colum
df2 = df2.withColumnRenamed('max_churn','label')

df2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: string (nullable = true)
 |-- max_location: string (nullable = true)
 |-- max_gender: string (nullable = true)
 |-- max_hasPaid: integer (nullable = true)
 |-- max_hasDowngrade: integer (nullable = true)
 |-- label: integer (nullable = true)
 |-- dist_artist: long (nullable = true)
 |-- avg_session_length: double (nullable = true)
 |-- avg_session_songs: double (nullable = true)
 |-- total_session_length: double (nullable = true)
 |-- total_songs_played: long (nullable = true)
 |-- total_sessions: long (nullable = true)
 |-- time_btw_sessions: double (nullable = true)

In [27]:
df2.persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[userId: string, max_location: string, max_gender: string, max_hasPaid: int, max_hasDowngrade: int, label: int, dist_artist: bigint, avg_session_length: double, avg_session_songs: double, total_session_length: double, total_songs_played: bigint, total_sessions: bigint, time_btw_sessions: double]

#### Split data into validation and test set

- validation set is used for crossvalidation e gridsearch
- test set is used to evaluate the model perfomance on 'unseen' data

In [28]:
# split data into validation and test set
validation, test = df2.randomSplit([0.9, 0.1], seed=42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### define functions to model training and model evaluation

In [35]:
def fit_model(model, paramGrid):
    """
    Build a pipeline and fit a model using CrossValidator and hyperparam optimization.
    Numerical features are Scaled with StandardScaler
    Categorical feature gender is transformed with StringIndexer
    
    Args:
    model (spark.ml.classification): The classifier algorithm
    paramGrid (ParamGridBuilder): A grid of hyperparameters used to optimize the model
    
    return:
    fitted_model: A cross validated fitted model
    
    """
    
    #cv = CountVectorizer(inputCol="artist_list", outputCol="TF")
    #location_index = StringIndexer(inputCol='max_location', outputCol="location_index", handleInvalid='skip')

    gender_index = StringIndexer(inputCol='max_gender', outputCol="gender_index", handleInvalid='skip')

    #scaler
    #"max_hasDowngrade","max_hasPaid",
    numerical = ["dist_artist", "avg_session_length",
                 "avg_session_songs","total_session_length","total_songs_played",
                 "total_sessions","time_btw_sessions"]

    numfeatures = VectorAssembler(inputCols=numerical, outputCol='NumFeatures')
    scaler = StandardScaler(inputCol='NumFeatures', outputCol="ScaledNumFeatures")

    assembler = VectorAssembler(inputCols=[#"TF",
                                           #"location_index",
                                           "gender_index",
                                           "ScaledNumFeatures"
                                          ], 
                                outputCol='features')

    pipeline = Pipeline(stages=[gender_index,
                                numfeatures, 
                                scaler, 
                                assembler,
                                model])

    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=MulticlassClassificationEvaluator(metricName='f1'),
                              numFolds=3)

    fitted_model = crossval.fit(validation)
    
    return fitted_model

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
def evaluate_results(results):
    """
    Evaluate the predited values VS real values
    
    Returns:
    score (float): F1-Score
    """
    score = MulticlassClassificationEvaluator(metricName='f1').evaluate(results)
    return score

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Test several models

#### Logistic Regression

In [37]:
############################
# Logistic Regression
############################
lr=  LogisticRegression(maxIter=10, regParam=0.0, elasticNetParam=0)

lr_grid = ParamGridBuilder() \
    .addGrid(lr.regParam,[0.0, 0.1]) \
    .build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
lr_model = fit_model(lr, lr_grid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
lr_model.avgMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.805940180178285, 0.6881400466262212]

In [40]:
score = evaluate_results(lr_model.transform(test))
print('RandomForest f1-score: ', score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RandomForest f1-score:  0.799620266077386

#### RandomForest

In [41]:
##########################
# RandomForestClassifier
##########################
rf = RandomForestClassifier()
rf_grid = ParamGridBuilder() \
    .addGrid(rf.maxDepth,[4,10,20])\
    .addGrid(rf.numTrees,[15,30])\
    .build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
rf_model = fit_model(rf, rf_grid)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
rf_model.avgMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.827248008596593, 0.8186762659102289, 0.8790844307953065, 0.8771778916096381, 0.8728414009509793, 0.8764687003416656]

In [44]:
score = evaluate_results(rf_model.transform(test))
print('RandomForest f1-score: ', score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RandomForest f1-score:  0.8729393583940597

#### LinearSVC

In [45]:
############
# LinearSVC
############
lsvc = LinearSVC()
lsvc_grid = ParamGridBuilder() \
    .addGrid(lsvc.maxIter,[10,20])\
    .addGrid(lsvc.regParam,[0.0,0.1])\
    .build()

lsvc_model = fit_model(lsvc, lsvc_grid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
lsvc_model.avgMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.6880135436724973, 0.6880135436724973, 0.6881400466262212, 0.6880135436724973]

In [47]:
score = evaluate_results(lsvc_model.transform(test))
print('Linear SVC f1-score: ', score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Linear SVC f1-score:  0.6797480399168581

Exception in thread cell_monitor-42:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1103

Exception in thread cell_monitor-45:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][sta

# Conclusion

## **Results on full dataset, run on Amazon EMR Cluster:**

F1-Score
- LogisticRegression = 0.799
- RandomForest = 0.872
- LinearSVC = 0.679
